In [10]:
import os
import numpy as np
import matplotlib
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scienceplots

In [2]:
path = "../HeuristicBaselines"

file_name_ep_data = "survival.csv"
file_name_action_data = "line_action_topo_data.csv"
file_name_env_config = 'env_config.json'
max_env_steps = 8064
size_case = 14

# Initialization of data lists
summarized_data = []
boxplot_data = []


for dir in os.listdir(path):   
    agent_dir = os.path.join(path,dir)
    if os.path.isdir(agent_dir) and (not dir.startswith('.')):
        if len(os.listdir(agent_dir)) == 1:
               agent_dir = os.path.join(agent_dir, os.listdir(agent_dir)[0])
        ep_data = pd.read_csv(os.path.join(agent_dir, file_name_ep_data))
        # print(ep_data)
        action_data = pd.read_csv(os.path.join(agent_dir, file_name_action_data))
        # print(action_data)
        with open(os.path.join(agent_dir, file_name_env_config), 'r') as file:
            env_config = json.load(file)
        # print(env_config)

        # Extract relevant data from env_config
        agent_type = env_config['agent_type']
        action_space = env_config['action_space']
        rules = env_config['rules']
        opponent = True if 'opponent_class' in env_config['grid2op_kwargs'] else False

        # Compute mean values from episode data
        mean_ts_survived = ep_data['survived'].mean()
        mean_max_exec_time = ep_data['max exec time'].mean()
        mean_exec_time = ep_data['mean exec time'].mean()
        
        # get data from action_data (which is all ts for which rho.max() > activation_threshold = 0.95)
        action_exec_time = action_data['agent_exec_time'].mean()
        ts_overloaded = len(action_data[action_data['rho']>1.0])/len(ep_data) # mean ts_activated/episode
        unique_actions = len(action_data[['action_sub', 'action_topo']].drop_duplicates())
        unique_line_danger = len(action_data.line_danger.unique())
        subs_changed = action_data.action_sub.unique()
        subs_changed = np.delete(subs_changed, np.where(subs_changed>size_case))
        n_subs_changed = len(subs_changed)
        max_topo_depth = action_data['sub_topo_depth'].max()

        # Add a row to the list
        summarized_data.append({
            'agent_type': agent_type,
            'opponent': opponent,
            'action space': action_space,
            'AT': rules['activation_threshold'],
            'line_reco': rules['line_reco'],
            'line_disc': rules['line_disc'],
            'reset_topo': rules['reset_topo'],
            'steps survived': round(mean_ts_survived / max_env_steps * 100,1),
            'steps overloaded': round(ts_overloaded / mean_ts_survived * 100,3),
            'execution time [ms]': round(mean_exec_time*1000, 3),
            'agent execution time [ms]': round(action_exec_time*1000, 3),
            'maximum topology depth': max_topo_depth,
            'unique actions': unique_actions,
            'unique lines in danger': unique_line_danger,
            'unqique subs changed': n_subs_changed,
            'substations changed': subs_changed,
            # 'max execution time [ms]': round(mean_max_exec_time*1000, 3)
        })

        # For boxplots:
        rule_label = (f"threshold={rules['activation_threshold']}, "
                      f"reco={rules['line_reco']}, "
                      f"disc={rules['line_disc']}, "
                      f"reset={rules['reset_topo']}")

        # Add data to boxplot_data
        for survived_value in ep_data['survived']:
            boxplot_data.append({
                'agent_type': agent_type,
                'opponent': opponent,
                'action space': action_space,
                'rules': rule_label,
                'survived': survived_value
            })

    # print(os.listdir(agent_dir))
# Create a new DataFrame from the collected rows
summary_table = pd.DataFrame(summarized_data)
summary_table = summary_table.sort_values(['opponent', 'agent_type', 'action space', 'line_reco', 'line_disc', 'reset_topo'], ascending=[True, True, True, True, True, False])

# Display the resulting table
display(summary_table)

,agent_type,opponent,action space,AT,line_reco,line_disc,reset_topo,steps survived,steps overloaded,execution time [ms],agent execution time [ms],maximum topology depth,unique actions,unique lines in danger,unqique subs changed,substations changed
34,DoNothing,False,medha,0.95,False,False,2.0,20.9,0.185,0.064,0.063,1,1,4,0,[]
6,DoNothing,False,medha,0.95,False,True,2.0,19.5,0.154,0.078,0.077,1,1,4,0,[]
25,DoNothing,False,medha,0.95,True,False,2.0,20.9,0.185,0.074,0.073,1,1,4,0,[]
7,DoNothing,False,medha,0.95,True,True,2.0,19.5,0.154,0.090,0.089,1,1,4,0,[]
13,RhoGreedy,False,medha,0.95,False,False,2.0,100.0,0.013,0.864,368.298,5,66,10,6,"[8, 3, 4, 1, 5, 12]"
1,RhoGreedy,False,medha,0.95,False,False,0.8,100.0,0.006,4.455,375.330,5,51,11,5,"[8, 3, 4, 1, 5]"
33,RhoGreedy,False,medha,0.95,False,True,2.0,100.0,0.013,0.879,370.531,5,66,10,6,"[8, 3, 4, 1, 5, 12]"
30,RhoGreedy,False,medha,0.95,False,True,0.8,100.0,0.006,4.555,382.306,5,51,11,5,"[8, 3, 4, 1, 5]"
9,RhoGreedy,False,medha,0.95,True,False,2.0,100.0,0.014,0.870,369.669,5,65,12,5,"[8, 3, 4, 1, 5]"
0,RhoGreedy,False,medha,0.95,True,False,0.8,100.0,0.006,4.464,374.968,5,51,11,5,"[8, 3, 4, 1, 5]"


In [3]:
display(action_data)

,chron_id,ts_danger,line_danger,rho,action_sub,action_topo,subs_changed,sub_topo_depth,el_changed,el_topo_depth,agent_exec_time,lines_disc
0,20,144,7,0.968443,3,"[1, 2, 2, 1, 1, 2]",[3],1,[14 15 18],3,0.210196,[19]
1,20,432,9,0.992893,8,"[1, 1, 2, 2, 2]",[8],1,[38 39 40],3,0.209581,[6]
2,20,463,17,0.982942,8,"[1, 1, 1, 1, 1]",[0],1,[],0,0.200949,[6]
3,20,464,17,0.950729,5,"[1, 2, 1, 2, 1, 1, 2]",[5],1,[25 27 30],3,0.206583,[ 6 13]
4,20,470,10,0.962572,8,"[1, 2, 2, 1, 1]",[5 8],2,[25 27 30 37 38],5,0.201825,[ 6 13]
...,...,...,...,...,...,...,...,...,...,...,...,...
2145,1000,1,17,0.956185,4,"[1, 2, 2, 1, 1]",[4],1,[20 21],2,0.210860,[16]
2146,1000,432,9,0.967607,5,"[1, 2, 1, 2, 2, 1, 2]",[5],1,[25 27 28 30],4,0.207760,[13]
2147,1000,456,10,0.990877,5,"[1, 1, 2, 2, 1, 1, 1]",[5],1,[26 27],2,0.207527,[13]
2148,1000,486,10,0.971759,5,"[1, 2, 1, 2, 1, 2, 2]",[5],1,[25 27 29 30],4,0.211469,[13]


In [4]:
data_with_opponent = summary_table[summary_table['opponent']]
display(data_with_opponent)

,agent_type,opponent,action space,AT,line_reco,line_disc,reset_topo,steps survived,steps overloaded,execution time [ms],agent execution time [ms],maximum topology depth,unique actions,unique lines in danger,unqique subs changed,substations changed
32,DoNothing,True,medha,0.95,False,False,2.0,1.9,4.137,0.065,0.062,1,1,12,0,[]
36,DoNothing,True,medha,0.95,False,True,2.0,1.8,4.417,0.111,0.143,1,1,12,0,[]
10,DoNothing,True,medha,0.95,True,False,2.0,4.6,1.337,0.140,0.241,1,1,12,0,[]
15,DoNothing,True,medha,0.95,True,True,2.0,5.2,2.039,0.224,1.144,1,1,11,0,[]
14,RhoGreedy,True,medha,0.95,False,False,2.0,4.7,1.490,37.444,368.653,5,79,15,6,"[1, 8, 4, 3, 5, 12]"
38,RhoGreedy,True,medha,0.95,False,False,0.8,6.9,1.422,41.199,366.913,5,87,17,6,"[3, 8, 4, 1, 5, 12]"
11,RhoGreedy,True,medha,0.95,False,True,2.0,4.8,0.848,35.933,357.556,5,81,15,6,"[1, 4, 5, 3, 8, 12]"
18,RhoGreedy,True,medha,0.95,False,True,0.8,7.1,0.550,41.229,364.148,5,73,14,6,"[4, 8, 1, 12, 3, 5]"
35,RhoGreedy,True,medha,0.95,True,False,2.0,9.0,1.217,29.594,365.313,5,88,14,5,"[5, 4, 8, 1, 3]"
4,RhoGreedy,True,medha,0.95,True,False,0.8,8.9,1.384,38.234,368.080,6,81,17,6,"[5, 8, 4, 3, 1, 12]"


In [5]:
# Convert boxplot_data into a DataFrame
boxplot_df = pd.DataFrame(boxplot_data)
boxplot_df = boxplot_df.sort_values(['opponent', 'agent_type'])
# # Filter only medha action space
# boxplot_df = boxplot_df[boxplot_df['action space']=='medha']

# Set up a color palette
palette = sns.color_palette("Set1", len(boxplot_df['rules'].unique()))
boxplot_df

,agent_type,opponent,action space,rules,survived
600,DoNothing,False,medha,"threshold=0.95, reco=False, disc=True, reset=2.0",519
601,DoNothing,False,medha,"threshold=0.95, reco=False, disc=True, reset=2.0",794
602,DoNothing,False,medha,"threshold=0.95, reco=False, disc=True, reset=2.0",3268
603,DoNothing,False,medha,"threshold=0.95, reco=False, disc=True, reset=2.0",1086
604,DoNothing,False,medha,"threshold=0.95, reco=False, disc=True, reset=2.0",521
...,...,...,...,...,...
3995,RhoGreedy,True,tennet,"threshold=0.95, reco=True, disc=True, reset=0.8",866
3996,RhoGreedy,True,tennet,"threshold=0.95, reco=True, disc=True, reset=0.8",473
3997,RhoGreedy,True,tennet,"threshold=0.95, reco=True, disc=True, reset=0.8",864
3998,RhoGreedy,True,tennet,"threshold=0.95, reco=True, disc=True, reset=0.8",1442


In [18]:
style = ['science', 'grid', 'no-latex', 'bright']
plt.style.use(style)
# # Create a new column combining agent_type and opponent for separate subplots
boxplot_df['agent_opponent_actspace'] = boxplot_df.apply(
    lambda row: f"{row['agent_type']} - "
                f"{row['action space']}" + (" with opponent" if row['opponent'] else f""), axis=1
)
# display(boxplot_df)

# Create a combined column for unique groups
boxplot_df['group'] = boxplot_df.apply(
    lambda row: f"{row['agent_type']} | {row['action space']} | "
                f"{row['rules']} | Opponent={row['opponent']}", axis=1
)

# Create the boxplots
plt.figure(figsize=(12, 8))
ax = plt.gca()

# Boxplot for survived by agent_type
boxplot = sns.boxplot(
    data=boxplot_df,
    x='agent_opponent_actspace',
    y='survived',
    hue='rules',
    palette=palette,
    ax=ax
)

# Apply hatching based on opponent status
patches = [patch for patch in boxplot.patches if type(patch) == mpatches.PathPatch]
for patch, (_, row) in zip(patches, boxplot_df[['group', 'opponent']].drop_duplicates().iterrows()):
    if row['opponent']:  # Apply hatching if opponent=True
        patch.set_hatch('//')

# Customize legend
handles, labels = ax.get_legend_handles_labels()
hatch_patch = mpatches.Patch(facecolor="white", edgecolor="black", hatch="//", label="Opponent=True")
plain_patch = mpatches.Patch(facecolor="white", edgecolor="black", label="Opponent=False")
ax.legend(
    handles=handles + [hatch_patch, plain_patch],
    labels=labels + ["Opponent=True", "Opponent=False"],
    title="Rules and Opponent",
    bbox_to_anchor=(1.05, 1),
    loc='upper left'
)

# Customize the plot
plt.title("Steps survived per agent", fontsize=14)
plt.xlabel("Agent Type", fontsize=12)
plt.ylabel("Survived", fontsize=12)
x_labels = [f"{row['agent_type']} - {'(N-0)' if row['action space'] == 'medha' else '(N-1)'}" for (_, row) in
            boxplot_df[['agent_opponent_actspace', 'agent_type', 'action space']].drop_duplicates().iterrows()]
plt.xticks(rotation=45)
boxplot.set_xticks([],minor=True)
boxplot.set_xticklabels(x_labels)
plt.tight_layout()

# Save the plot
plt.savefig(os.path.join(path, f'box_plots_agents_rules_opponent.svg'))

In [7]:
patches = [patch for patch in boxplot.patches if type(patch) == mpatches.PathPatch]
len(patches)

40

In [11]:
plt.style.available

['Solarize_Light2',
 '_classic_test_patch',
 '_mpl-gallery',
 '_mpl-gallery-nogrid',
 'bmh',
 'bright',
 'cjk-jp-font',
 'cjk-kr-font',
 'cjk-sc-font',
 'cjk-tc-font',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'grid',
 'high-contrast',
 'high-vis',
 'ieee',
 'latex-sans',
 'light',
 'muted',
 'nature',
 'no-latex',
 'notebook',
 'pgf',
 'retro',
 'russian-font',
 'sans',
 'scatter',
 'science',
 'seaborn',
 'seaborn-bright',
 'seaborn-colorblind',
 'seaborn-dark',
 'seaborn-dark-palette',
 'seaborn-darkgrid',
 'seaborn-deep',
 'seaborn-muted',
 'seaborn-notebook',
 'seaborn-paper',
 'seaborn-pastel',
 'seaborn-poster',
 'seaborn-talk',
 'seaborn-ticks',
 'seaborn-white',
 'seaborn-whitegrid',
 'std-colors',
 'tableau-colorblind10',
 'turkish-font',
 'vibrant']

In [8]:
boxplot_df['group'] = boxplot_df.apply(
    lambda row: f"{row['agent_type']} | {row['rules']} | Opponent={row['opponent']}", axis=1
)
for i, (_,row) in enumerate(boxplot_df[['group', 'opponent']].drop_duplicates().iterrows()):
    print(row['opponent'])

False
False
True
True
False
False
False
True
True
False
False
True
True
True
False
False
False
True
False
False
True
True
True
True


In [40]:
len(boxplot_df[['group', 'opponent']].drop_duplicates())

24

In [17]:
[row['agent_type'] for (_,row) in boxplot_df[['agent_opponent', 'agent_type']].drop_duplicates().iterrows()]

['DoNothing', 'RhoGreedy', 'DoNothing', 'RhoGreedy']